# Model Training (All Pairs)

Multi-branch neural model with influence-controlled news embedding.

**Difference from 2.0**: This version generates ALL possible pairs per day rather than sampling. With ~160 stocks/day with news, that's ~12,700 pairs/day.

**Table of Contents:**
1. Load and split data
2. Dataset and DataLoader
3. Model architecture
4. Training loop
5. Evaluation: Rank IC and Basket Returns

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [2]:
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelConfig:
    # Feature dimensions (set after loading data)
    n_fundamental_features: int = 19
    n_price_features: int = 9
    n_embedding_dim: int = 768
    
    # Encoder latent dimensions
    fundamental_latent: int = 32
    price_latent: int = 16
    news_latent: int = 32
    
    # Dropout
    fundamental_dropout: float = 0.2
    price_dropout: float = 0.2
    news_dropout: float = 0.3
    
    # News influence cap
    news_alpha: float = 0.8
    
    # Training
    batch_size: int = 512
    learning_rate: float = 1e-3
    weight_decay: float = 1e-3
    n_epochs: int = 20
    pairs_per_day: int = 5000

    # Random pairs only (no hard pair oversampling)
    hard_fraction: float = 0.0

## 1. Load and split data

In [3]:
df = pd.read_parquet("data/ml_dataset.pqt")
df["feature_date"] = pd.to_datetime(df["feature_date"])

print(f"Dataset: {len(df):,} rows")
print(f"Date range: {df['feature_date'].min().date()} to {df['feature_date'].max().date()}")
print(f"Symbols: {df['symbol'].nunique():,}")

Dataset: 2,092,929 rows
Date range: 2021-01-13 to 2025-12-18
Symbols: 3,506


In [4]:
# Identify feature columns
price_feat_cols = [
    "overnight_gap_z", "intraday_ret_z",
    "ret_1d_z", "ret_2d_z", "ret_3d_z", "ret_5d_z",
    "vol_5d_z", "dist_from_high_5d_z", "dist_from_low_5d_z"
]

# Fundamental feature columns (normalized)
fund_feat_cols = [c for c in df.columns if c.endswith("_z") and c not in price_feat_cols and c != "news_count_z"]

# Embedding columns
emb_cols = [c for c in df.columns if c.startswith("emb_")]

print(f"Price features: {len(price_feat_cols)}")
print(f"Fundamental features: {len(fund_feat_cols)}")
print(f"Embedding dims: {len(emb_cols)}")

Price features: 9
Fundamental features: 19
Embedding dims: 768


In [5]:
# Time-based split (per README section 13)
# Use last 20% of dates as test
dates = sorted(df["feature_date"].unique())
n_dates = len(dates)
train_end_idx = int(n_dates * 0.7)
val_end_idx = int(n_dates * 0.85)

train_dates = set(dates[:train_end_idx])
val_dates = set(dates[train_end_idx:val_end_idx])
test_dates = set(dates[val_end_idx:])

train_df = df[df["feature_date"].isin(train_dates)].copy()
val_df = df[df["feature_date"].isin(val_dates)].copy()
test_df = df[df["feature_date"].isin(test_dates)].copy()

print(f"Train: {len(train_df):,} rows, {len(train_dates)} days")
print(f"Val: {len(val_df):,} rows, {len(val_dates)} days")
print(f"Test: {len(test_df):,} rows, {len(test_dates)} days")

Train: 1,418,494 rows, 830 days
Val: 322,222 rows, 178 days
Test: 352,213 rows, 178 days


## 2. Dataset and DataLoader

In [6]:
class PairwiseRankingDataset(Dataset):
    """Dataset that generates ALL pairs from same day for ranking loss.

    Only uses rows with news. Generates all N*(N-1)/2 unique pairs per day.
    
    Call resample_pairs() at the start of each epoch to shuffle the order.
    """

    def __init__(
        self,
        df: pd.DataFrame,
        price_cols: list[str],
        fund_cols: list[str],
        emb_cols: list[str],
    ):
        # Filter to rows with news only
        has_news = (df[emb_cols].abs().sum(axis=1) > 0)
        df_news = df[has_news].copy()
        print(f"Filtered to news-only: {len(df_news):,} / {len(df):,} rows ({len(df_news)/len(df)*100:.1f}%)")

        self.df = df_news.reset_index(drop=True)
        self.price_cols = price_cols
        self.fund_cols = fund_cols
        self.emb_cols = emb_cols

        # Group by date
        self.date_groups = {}
        for date, group in df_news.groupby("feature_date"):
            indices = group.index.tolist()
            if len(indices) < 2:  # Need at least 2 for pairs
                continue
            self.date_groups[date] = np.array(indices)

        self.dates = list(self.date_groups.keys())
        print(f"Days with sufficient news coverage: {len(self.dates)}")

        # Precompute arrays for speed
        self.price_arr = df_news[price_cols].values.astype(np.float32)
        self.fund_arr = df_news[fund_cols].values.astype(np.float32)
        self.emb_arr = df_news[emb_cols].values.astype(np.float32)
        self.target_arr = df_news["target_return"].values.astype(np.float32)

        # Map original index to position in filtered df
        self.idx_map = {old_idx: new_idx for new_idx, old_idx in enumerate(df_news.index)}

        # Generate all pairs
        self.pairs = []
        self._generate_all_pairs()
        
    def _generate_all_pairs(self):
        """Generate ALL unique pairs for each day."""
        pairs = []
        total_possible = 0

        for date in self.dates:
            indices = self.date_groups[date]
            n = len(indices)
            total_possible += n * (n - 1) // 2
            
            # Generate all unique pairs (i, j) where i < j
            for i in range(n):
                for j in range(i + 1, n):
                    idx_i = self.idx_map[indices[i]]
                    idx_j = self.idx_map[indices[j]]
                    pairs.append((idx_i, idx_j))

        self.pairs = pairs
        print(f"Generated {len(self.pairs):,} pairs (all combinations)")
        
    def resample_pairs(self):
        """Shuffle pairs for new epoch (no resampling needed since we use all)."""
        np.random.shuffle(self.pairs)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        i, j = self.pairs[idx]

        # Get features
        price_i = self.price_arr[i]
        price_j = self.price_arr[j]
        fund_i = self.fund_arr[i]
        fund_j = self.fund_arr[j]
        emb_i = self.emb_arr[i]
        emb_j = self.emb_arr[j]

        # Label based on actual returns
        actual_label = 1.0 if self.target_arr[i] > self.target_arr[j] else 0.0

        # Randomly swap to balance labels
        if np.random.random() < 0.5:
            price_i, price_j = price_j, price_i
            fund_i, fund_j = fund_j, fund_i
            emb_i, emb_j = emb_j, emb_i
            label = 1.0 - actual_label
        else:
            label = actual_label

        return {
            "price_i": torch.tensor(price_i),
            "price_j": torch.tensor(price_j),
            "fund_i": torch.tensor(fund_i),
            "fund_j": torch.tensor(fund_j),
            "emb_i": torch.tensor(emb_i),
            "emb_j": torch.tensor(emb_j),
            "label": torch.tensor(label),
        }

In [7]:
class PointwiseDataset(Dataset):
    """Dataset for inference - one sample per stock-day."""
    
    def __init__(
        self,
        df: pd.DataFrame,
        price_cols: list[str],
        fund_cols: list[str],
        emb_cols: list[str],
    ):
        self.df = df.reset_index(drop=True)
        self.price_arr = df[price_cols].values.astype(np.float32)
        self.fund_arr = df[fund_cols].values.astype(np.float32)
        self.emb_arr = df[emb_cols].values.astype(np.float32)
        self.target_arr = df["target_return"].values.astype(np.float32)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        return {
            "price": torch.tensor(self.price_arr[idx]),
            "fund": torch.tensor(self.fund_arr[idx]),
            "emb": torch.tensor(self.emb_arr[idx]),
            "target": torch.tensor(self.target_arr[idx]),
        }

In [8]:
# Create config with actual dimensions
config = ModelConfig(
    n_fundamental_features=len(fund_feat_cols),
    n_price_features=len(price_feat_cols),
    n_embedding_dim=len(emb_cols),
)

print(f"Config: {config}")

Config: ModelConfig(n_fundamental_features=19, n_price_features=9, n_embedding_dim=768, fundamental_latent=32, price_latent=16, news_latent=32, fundamental_dropout=0.2, price_dropout=0.2, news_dropout=0.3, news_alpha=0.8, batch_size=512, learning_rate=0.001, weight_decay=0.001, n_epochs=20, pairs_per_day=5000, hard_fraction=0.0)


In [9]:
# Create datasets with ALL pairs
train_dataset = PairwiseRankingDataset(
    train_df, price_feat_cols, fund_feat_cols, emb_cols,
)
val_dataset = PairwiseRankingDataset(
    val_df, price_feat_cols, fund_feat_cols, emb_cols,
)

# For evaluation (pointwise)
val_pointwise = PointwiseDataset(val_df, price_feat_cols, fund_feat_cols, emb_cols)
test_pointwise = PointwiseDataset(test_df, price_feat_cols, fund_feat_cols, emb_cols)

print(f"\nTrain pairs: {len(train_dataset):,}")
print(f"Val pairs: {len(val_dataset):,}")

Filtered to news-only: 125,880 / 1,418,494 rows (8.9%)
Days with sufficient news coverage: 830
Generated 9,742,161 pairs (all combinations)
Filtered to news-only: 32,360 / 322,222 rows (10.0%)
Days with sufficient news coverage: 178
Generated 2,981,143 pairs (all combinations)

Train pairs: 9,742,161
Val pairs: 2,981,143


In [10]:
# Create dataloaders
# num_workers=0 for Jupyter compatibility (avoids multiprocessing issues)
train_loader = DataLoader(
    train_dataset, 
    batch_size=config.batch_size, 
    shuffle=True,
    num_workers=0,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=0,
)

## 3. Model architecture

In [11]:
class MultiBranchRanker(nn.Module):
    """Multi-branch model with influence-controlled news embedding."""
    
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.config = config
        
        # Fundamentals encoder
        self.fund_encoder = nn.Sequential(
            nn.Linear(config.n_fundamental_features, 64),
            nn.ReLU(),
            nn.Dropout(config.fundamental_dropout),
            nn.Linear(64, config.fundamental_latent),
            nn.ReLU(),
        )
        
        # Price encoder
        self.price_encoder = nn.Sequential(
            nn.Linear(config.n_price_features, 32),
            nn.ReLU(),
            nn.Dropout(config.price_dropout),
            nn.Linear(32, config.price_latent),
            nn.ReLU(),
        )
        
        # News encoder
        self.news_encoder = nn.Sequential(
            nn.Linear(config.n_embedding_dim, 128),
            nn.ReLU(),
            nn.Dropout(config.news_dropout),
            nn.Linear(128, config.news_latent),
            nn.ReLU(),
        )
        
        # Output head
        fused_dim = config.fundamental_latent + config.price_latent + config.news_latent
        self.output_head = nn.Sequential(
            nn.Linear(fused_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(32, 1),
        )
    
    def encode(self, price: torch.Tensor, fund: torch.Tensor, emb: torch.Tensor) -> torch.Tensor:
        """Encode features and return fused representation."""
        h_f = self.fund_encoder(fund)
        h_p = self.price_encoder(price)
        h_n = self.news_encoder(emb)
        
        # Apply news influence cap (α * h_n)
        h_n_scaled = self.config.news_alpha * h_n
        
        # Fuse
        h = torch.cat([h_f, h_p, h_n_scaled], dim=-1)
        return h
    
    def forward(self, price: torch.Tensor, fund: torch.Tensor, emb: torch.Tensor) -> torch.Tensor:
        """Forward pass returning score."""
        h = self.encode(price, fund, emb)
        score = self.output_head(h)
        return score.squeeze(-1)
    
    def forward_pair(
        self,
        price_i: torch.Tensor, fund_i: torch.Tensor, emb_i: torch.Tensor,
        price_j: torch.Tensor, fund_j: torch.Tensor, emb_j: torch.Tensor,
    ) -> torch.Tensor:
        """Forward pass for pair, returning P(i > j)."""
        score_i = self.forward(price_i, fund_i, emb_i)
        score_j = self.forward(price_j, fund_j, emb_j)
        return torch.sigmoid(score_i - score_j)

In [12]:
model = MultiBranchRanker(config).to(device)

# Count parameters
n_params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {n_params:,}")
print(model)

Model parameters: 109,393
MultiBranchRanker(
  (fund_encoder): Sequential(
    (0): Linear(in_features=19, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=64, out_features=32, bias=True)
    (4): ReLU()
  )
  (price_encoder): Sequential(
    (0): Linear(in_features=9, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): ReLU()
  )
  (news_encoder): Sequential(
    (0): Linear(in_features=768, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=128, out_features=32, bias=True)
    (4): ReLU()
  )
  (output_head): Sequential(
    (0): Linear(in_features=80, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=32, out_features=1, bias=True)
  )
)


## 4. Training loop

In [13]:
def pairwise_ranking_loss(pred_prob: torch.Tensor, label: torch.Tensor, smoothing: float = 0.2) -> torch.Tensor:
    """Binary cross-entropy for pairwise ranking with label smoothing."""
    # Smooth labels: 0 -> smoothing/2, 1 -> 1 - smoothing/2
    smoothed_label = label * (1 - smoothing) + 0.5 * smoothing
    return F.binary_cross_entropy(pred_prob, smoothed_label)

In [14]:
def train_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    for batch in tqdm(loader, desc="Training", leave=False):
        price_i = batch["price_i"].to(device)
        price_j = batch["price_j"].to(device)
        fund_i = batch["fund_i"].to(device)
        fund_j = batch["fund_j"].to(device)
        emb_i = batch["emb_i"].to(device)
        emb_j = batch["emb_j"].to(device)
        label = batch["label"].to(device)
        
        optimizer.zero_grad()
        pred_prob = model.forward_pair(price_i, fund_i, emb_i, price_j, fund_j, emb_j)
        loss = pairwise_ranking_loss(pred_prob, label)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * len(label)
        total_correct += ((pred_prob > 0.5) == (label > 0.5)).sum().item()
        total_samples += len(label)
    
    return total_loss / total_samples, total_correct / total_samples

In [15]:
@torch.no_grad()
def eval_epoch(model, loader, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    for batch in tqdm(loader, desc="Evaluating", leave=False):
        price_i = batch["price_i"].to(device)
        price_j = batch["price_j"].to(device)
        fund_i = batch["fund_i"].to(device)
        fund_j = batch["fund_j"].to(device)
        emb_i = batch["emb_i"].to(device)
        emb_j = batch["emb_j"].to(device)
        label = batch["label"].to(device)
        
        pred_prob = model.forward_pair(price_i, fund_i, emb_i, price_j, fund_j, emb_j)
        loss = pairwise_ranking_loss(pred_prob, label)
        
        total_loss += loss.item() * len(label)
        total_correct += ((pred_prob > 0.5) == (label > 0.5)).sum().item()
        total_samples += len(label)
    
    return total_loss / total_samples, total_correct / total_samples

In [16]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config.learning_rate,
    weight_decay=config.weight_decay,
)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=config.n_epochs,
)

In [19]:
torch.save(model.state_dict(), "data/model_best_all_pairs_ep3.pt")

In [18]:
# Training
best_val_acc = 0
history = []

for epoch in range(config.n_epochs):
    # Resample pairs each epoch for fresh training data
    train_dataset.resample_pairs()
    val_dataset.resample_pairs()
    
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, device)
    val_loss, val_acc = eval_epoch(model, val_loader, device)
    scheduler.step()
    
    history.append({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "val_loss": val_loss,
        "val_acc": val_acc,
    })
    
    print(f"Epoch {epoch+1}/{config.n_epochs}: "
          f"train_loss={train_loss:.4f}, train_acc={train_acc:.4f}, "
          f"val_loss={val_loss:.4f}, val_acc={val_acc:.4f}")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "data/model_best_all_pairs.pt")
        print(f"  -> New best model saved")

Training:   0%|          | 0/19028 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/5823 [00:00<?, ?it/s]

Epoch 1/20: train_loss=0.6823, train_acc=0.5603, val_loss=0.7039, val_acc=0.5036
  -> New best model saved


Training:   0%|          | 0/19028 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/5823 [00:00<?, ?it/s]

...

Training:   0%|          | 0/19028 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/5823 [00:00<?, ?it/s]

Epoch 3/20: train_loss=0.6569, train_acc=0.6224, val_loss=0.7219, val_acc=0.5020


Training:   0%|          | 0/19028 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 5. Evaluation: Rank IC and Basket Returns

In [ ]:
# Load best model
model.load_state_dict(torch.load("data/model_best.pt", weights_only=True))
model.eval()

In [ ]:
@torch.no_grad()
def get_scores(model, df, price_cols, fund_cols, emb_cols, device, batch_size=1024):
    """Get model scores for all rows."""
    model.eval()
    dataset = PointwiseDataset(df, price_cols, fund_cols, emb_cols)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    scores = []
    for batch in tqdm(loader, desc="Scoring", leave=False):
        price = batch["price"].to(device)
        fund = batch["fund"].to(device)
        emb = batch["emb"].to(device)
        
        score = model(price, fund, emb)
        scores.append(score.cpu().numpy())
    
    return np.concatenate(scores)

In [ ]:
# Score test set
test_df = test_df.copy()
test_df["score"] = get_scores(model, test_df, price_feat_cols, fund_feat_cols, emb_cols, device)

print(f"Scored {len(test_df):,} test samples")

In [ ]:
from scipy.stats import spearmanr

def compute_daily_ic(df):
    """Compute Spearman rank IC per day."""
    ics = []
    for date, group in df.groupby("feature_date"):
        if len(group) < 10:
            continue
        ic, _ = spearmanr(group["score"], group["target_return"])
        if not np.isnan(ic):
            ics.append({"date": date, "ic": ic})
    return pd.DataFrame(ics)

In [ ]:
ic_df = compute_daily_ic(test_df)

print(f"Daily Rank IC (Spearman):")
print(f"  Mean: {ic_df['ic'].mean():.4f}")
print(f"  Std: {ic_df['ic'].std():.4f}")
print(f"  Sharpe (IC): {ic_df['ic'].mean() / ic_df['ic'].std() * np.sqrt(252):.2f}")
print(f"  % Positive: {(ic_df['ic'] > 0).mean()*100:.1f}%")

In [ ]:
def compute_basket_returns(df, top_k=20):
    """Compute daily returns of top-K basket (equal weight)."""
    returns = []
    for date, group in df.groupby("feature_date"):
        if len(group) < top_k:
            continue
        # Select top-K by score
        top = group.nlargest(top_k, "score")
        # Equal weight return
        ret = top["target_return"].mean()
        # Market return (all stocks)
        mkt_ret = group["target_return"].mean()
        returns.append({
            "date": date,
            "basket_return": ret,
            "market_return": mkt_ret,
            "excess_return": ret - mkt_ret,
        })
    return pd.DataFrame(returns)

In [ ]:
basket_df = compute_basket_returns(test_df, top_k=20)

print(f"\nTop-20 Basket Performance (Test Set):")
print(f"  Daily mean return: {basket_df['basket_return'].mean()*100:.3f}%")
print(f"  Daily mean excess: {basket_df['excess_return'].mean()*100:.3f}%")
print(f"  Sharpe (basket): {basket_df['basket_return'].mean() / basket_df['basket_return'].std() * np.sqrt(252):.2f}")
print(f"  Sharpe (excess): {basket_df['excess_return'].mean() / basket_df['excess_return'].std() * np.sqrt(252):.2f}")

In [ ]:
# Cumulative returns
basket_df["cum_basket"] = (1 + basket_df["basket_return"]).cumprod()
basket_df["cum_market"] = (1 + basket_df["market_return"]).cumprod()
basket_df["cum_excess"] = (1 + basket_df["excess_return"]).cumprod()

print(f"\nCumulative Returns:")
print(f"  Basket: {(basket_df['cum_basket'].iloc[-1] - 1)*100:.1f}%")
print(f"  Market: {(basket_df['cum_market'].iloc[-1] - 1)*100:.1f}%")
print(f"  Excess: {(basket_df['cum_excess'].iloc[-1] - 1)*100:.1f}%")

In [ ]:
# Long-short analysis: long top-K, short bottom-K
def compute_long_short_returns(df, top_k=20):
    """Compute daily returns of long-short portfolio."""
    returns = []
    for date, group in df.groupby("feature_date"):
        if len(group) < top_k * 2:
            continue
        # Long top-K
        top = group.nlargest(top_k, "score")
        long_ret = top["target_return"].mean()
        # Short bottom-K
        bottom = group.nsmallest(top_k, "score")
        short_ret = bottom["target_return"].mean()
        # Long-short return (long winners, short losers)
        ls_ret = long_ret - short_ret
        # Market
        mkt_ret = group["target_return"].mean()
        returns.append({
            "date": date,
            "long_return": long_ret,
            "short_return": short_ret,
            "long_short_return": ls_ret,
            "market_return": mkt_ret,
        })
    return pd.DataFrame(returns)

ls_df = compute_long_short_returns(test_df, top_k=20)

print("Long-Short Portfolio Performance (Test Set):")
print(f"\n  Long (top-20):")
print(f"    Daily mean: {ls_df['long_return'].mean()*100:.3f}%")
print(f"    Cumulative: {((1 + ls_df['long_return']).cumprod().iloc[-1] - 1)*100:.1f}%")

print(f"\n  Short (bottom-20):")
print(f"    Daily mean: {ls_df['short_return'].mean()*100:.3f}%")
print(f"    Cumulative: {((1 + ls_df['short_return']).cumprod().iloc[-1] - 1)*100:.1f}%")
print(f"    Short P&L (inverted): {((1 - ls_df['short_return']).cumprod().iloc[-1] - 1)*100:.1f}%")

print(f"\n  Long-Short (market neutral):")
print(f"    Daily mean: {ls_df['long_short_return'].mean()*100:.3f}%")
print(f"    Std: {ls_df['long_short_return'].std()*100:.3f}%")
print(f"    Sharpe: {ls_df['long_short_return'].mean() / ls_df['long_short_return'].std() * np.sqrt(252):.2f}")
print(f"    Cumulative: {((1 + ls_df['long_short_return']).cumprod().iloc[-1] - 1)*100:.1f}%")

print(f"\n  Market:")
print(f"    Cumulative: {((1 + ls_df['market_return']).cumprod().iloc[-1] - 1)*100:.1f}%")

In [ ]:
# Investigate the short basket - are these real shortable stocks or distressed junk?
print("Bottom-20 basket analysis:")
print("=" * 50)

# Get all bottom-20 selections
bottom_picks = []
for date, group in test_df.groupby("feature_date"):
    if len(group) < 40:
        continue
    bottom = group.nsmallest(20, "score")
    bottom_picks.append(bottom[["symbol", "feature_date", "target_return", "score"]])

bottom_df = pd.concat(bottom_picks)

print(f"\nMost frequent bottom-20 stocks:")
freq = bottom_df["symbol"].value_counts().head(20)
print(freq)

print(f"\n\nWorst single-day returns in bottom basket:")
print(bottom_df.nsmallest(20, "target_return")[["feature_date", "symbol", "target_return"]])

print(f"\n\nReturn distribution of bottom-20 picks:")
print(bottom_df["target_return"].describe())

# Check for extreme outliers driving the result
print(f"\n\nDays with extreme short returns (< -10%):")
extreme_days = ls_df[ls_df["short_return"] < -0.10]
print(f"Number of extreme days: {len(extreme_days)}")
if len(extreme_days) > 0:
    print(extreme_days[["date", "short_return", "long_return", "long_short_return"]].head(10))

In [ ]:
# Check for data quality issues - returns < -100% are impossible
print("Data quality check:")
print("=" * 50)
impossible_returns = test_df[test_df["target_return"] < -1.0]
print(f"\nRows with impossible returns (< -100%): {len(impossible_returns)}")
if len(impossible_returns) > 0:
    print(impossible_returns[["symbol", "feature_date", "target_return"]].head(20))

# Also check extreme positive returns
extreme_positive = test_df[test_df["target_return"] > 1.0]
print(f"\nRows with extreme positive returns (> 100%): {len(extreme_positive)}")
if len(extreme_positive) > 0:
    print(extreme_positive[["symbol", "feature_date", "target_return"]].head(10))

In [ ]:
# Re-evaluate with winsorized returns (clip extreme values)
def compute_long_short_returns_clipped(df, top_k=20, clip_pct=0.50):
    """Compute long-short returns with clipped extreme returns."""
    df = df.copy()
    df["target_return_clipped"] = df["target_return"].clip(-clip_pct, clip_pct)
    
    returns = []
    for date, group in df.groupby("feature_date"):
        if len(group) < top_k * 2:
            continue
        # Use original score for ranking, clipped returns for P&L
        top = group.nlargest(top_k, "score")
        long_ret = top["target_return_clipped"].mean()
        
        bottom = group.nsmallest(top_k, "score")
        short_ret = bottom["target_return_clipped"].mean()
        
        ls_ret = long_ret - short_ret
        mkt_ret = group["target_return_clipped"].mean()
        
        returns.append({
            "date": date,
            "long_return": long_ret,
            "short_return": short_ret,
            "long_short_return": ls_ret,
            "market_return": mkt_ret,
        })
    return pd.DataFrame(returns)

ls_clipped = compute_long_short_returns_clipped(test_df, top_k=20, clip_pct=0.50)

print("Long-Short with clipped returns (max ±50%):")
print("=" * 50)
print(f"\n  Long (top-20):")
print(f"    Cumulative: {((1 + ls_clipped['long_return']).cumprod().iloc[-1] - 1)*100:.1f}%")

print(f"\n  Short (bottom-20):")
print(f"    Cumulative: {((1 + ls_clipped['short_return']).cumprod().iloc[-1] - 1)*100:.1f}%")
print(f"    Short P&L: {((1 - ls_clipped['short_return']).cumprod().iloc[-1] - 1)*100:.1f}%")

print(f"\n  Long-Short:")
print(f"    Daily mean: {ls_clipped['long_short_return'].mean()*100:.3f}%")
print(f"    Sharpe: {ls_clipped['long_short_return'].mean() / ls_clipped['long_short_return'].std() * np.sqrt(252):.2f}")
print(f"    Cumulative: {((1 + ls_clipped['long_short_return']).cumprod().iloc[-1] - 1)*100:.1f}%")

### 5.4 Shortability Analysis

Filter to stocks that are practically shortable:
- Market cap > $500M (institutional threshold)
- Exclude penny stocks and micro-caps

In [ ]:
# Load fundamentals to get market cap
key_metrics = pd.read_parquet("data/key_metrics.pqt")

# Get latest market cap per symbol (most recent quarter)
key_metrics["date"] = pd.to_datetime(key_metrics["date"])
latest_mcap = (
    key_metrics[["symbol", "date", "marketCap"]]
    .sort_values("date")
    .groupby("symbol")
    .last()
    .reset_index()
)

print(f"Market cap data for {len(latest_mcap):,} symbols")
print(f"\nMarket cap distribution:")
print(latest_mcap["marketCap"].describe())

# Define market cap tiers
def mcap_tier(mcap):
    if pd.isna(mcap) or mcap <= 0:
        return "Unknown"
    elif mcap < 300_000_000:
        return "Micro (<$300M)"
    elif mcap < 2_000_000_000:
        return "Small ($300M-$2B)"
    elif mcap < 10_000_000_000:
        return "Mid ($2B-$10B)"
    else:
        return "Large (>$10B)"

latest_mcap["tier"] = latest_mcap["marketCap"].apply(mcap_tier)
print(f"\nMarket cap tiers:")
print(latest_mcap["tier"].value_counts())

In [ ]:
# Merge market cap into test_df
test_df_mcap = test_df.merge(
    latest_mcap[["symbol", "marketCap"]], 
    on="symbol", 
    how="left"
)

# Check how many of our bottom-20 picks have market cap data
bottom_with_mcap = bottom_df.merge(latest_mcap[["symbol", "marketCap"]], on="symbol", how="left")
print("Bottom-20 picks by market cap tier:")
bottom_with_mcap["tier"] = bottom_with_mcap["marketCap"].apply(mcap_tier)
print(bottom_with_mcap["tier"].value_counts())

print("\n\nMost frequent bottom-20 stocks with their market caps:")
freq_symbols = bottom_df["symbol"].value_counts().head(20).index.tolist()
for sym in freq_symbols:
    mcap = latest_mcap[latest_mcap["symbol"] == sym]["marketCap"].values
    mcap_str = f"${mcap[0]/1e9:.1f}B" if len(mcap) > 0 and mcap[0] > 0 else "N/A"
    count = bottom_df[bottom_df["symbol"] == sym].shape[0]
    print(f"  {sym}: {count} days, mcap={mcap_str}")

In [ ]:
# Re-evaluate long-short with market cap filter
MIN_MCAP = 500_000_000  # $500M minimum

# Filter to shortable universe
shortable_symbols = set(latest_mcap[latest_mcap["marketCap"] >= MIN_MCAP]["symbol"])
test_shortable = test_df_mcap[test_df_mcap["symbol"].isin(shortable_symbols)].copy()

print(f"Shortable universe (mcap >= ${MIN_MCAP/1e6:.0f}M): {len(shortable_symbols):,} symbols")
print(f"Test rows: {len(test_df_mcap):,} -> {len(test_shortable):,} ({len(test_shortable)/len(test_df_mcap)*100:.1f}%)")

# Re-run long-short on shortable universe
def compute_long_short_returns_filtered(df, top_k=20, clip_pct=0.50):
    """Long-short with clipped returns on filtered universe."""
    df = df.copy()
    df["target_return_clipped"] = df["target_return"].clip(-clip_pct, clip_pct)
    
    returns = []
    for date, group in df.groupby("feature_date"):
        if len(group) < top_k * 2:
            continue
        
        top = group.nlargest(top_k, "score")
        long_ret = top["target_return_clipped"].mean()
        
        bottom = group.nsmallest(top_k, "score")
        short_ret = bottom["target_return_clipped"].mean()
        
        ls_ret = long_ret - short_ret
        mkt_ret = group["target_return_clipped"].mean()
        
        returns.append({
            "date": date,
            "long_return": long_ret,
            "short_return": short_ret,
            "long_short_return": ls_ret,
            "market_return": mkt_ret,
        })
    return pd.DataFrame(returns)

ls_shortable = compute_long_short_returns_filtered(test_shortable, top_k=20, clip_pct=0.50)

print(f"\nLong-Short on Shortable Universe (mcap >= $500M):")
print("=" * 50)
print(f"\n  Trading days: {len(ls_shortable)}")

print(f"\n  Long (top-20):")
print(f"    Cumulative: {((1 + ls_shortable['long_return']).cumprod().iloc[-1] - 1)*100:.1f}%")

print(f"\n  Short (bottom-20):")
print(f"    Cumulative: {((1 + ls_shortable['short_return']).cumprod().iloc[-1] - 1)*100:.1f}%")
print(f"    Short P&L: {((1 - ls_shortable['short_return']).cumprod().iloc[-1] - 1)*100:.1f}%")

print(f"\n  Long-Short:")
print(f"    Daily mean: {ls_shortable['long_short_return'].mean()*100:.3f}%")
print(f"    Sharpe: {ls_shortable['long_short_return'].mean() / ls_shortable['long_short_return'].std() * np.sqrt(252):.2f}")
print(f"    Cumulative: {((1 + ls_shortable['long_short_return']).cumprod().iloc[-1] - 1)*100:.1f}%")
print(f"    Win rate: {(ls_shortable['long_short_return'] > 0).mean()*100:.1f}%")

In [ ]:
# What stocks end up in the shortable bottom-20?
bottom_shortable = []
for date, group in test_shortable.groupby("feature_date"):
    if len(group) < 40:
        continue
    bottom = group.nsmallest(20, "score")
    bottom_shortable.append(bottom[["symbol", "feature_date", "target_return", "score", "marketCap"]])

bottom_shortable_df = pd.concat(bottom_shortable)

print("Most frequent bottom-20 stocks (shortable universe):")
freq = bottom_shortable_df["symbol"].value_counts().head(20)
for sym, count in freq.items():
    mcap = bottom_shortable_df[bottom_shortable_df["symbol"] == sym]["marketCap"].iloc[0]
    print(f"  {sym}: {count} days, mcap=${mcap/1e9:.1f}B")

print(f"\n\nReturn distribution of shortable bottom-20:")
print(bottom_shortable_df["target_return"].describe())

In [ ]:
# Save model and config
torch.save({
    "model_state_dict": model.state_dict(),
    "config": config,
    "price_cols": price_feat_cols,
    "fund_cols": fund_feat_cols,
    "emb_cols": emb_cols,
}, "data/model_final.pt")

print("Model saved to data/model_final.pt")

In [ ]:
# Training history
history_df = pd.DataFrame(history)
history_df

## 6. News Ablation Analysis

Evaluate the incremental value of news embeddings by comparing:
1. Model performance on rows with vs without news
2. Full model vs ablated model (zeroed news embeddings)
3. Statistical significance of the news signal

In [ ]:
# Identify rows with news (non-zero embeddings)
test_df["has_news"] = (test_df[emb_cols].abs().sum(axis=1) > 0).astype(int)
print(f"Test set news coverage: {test_df['has_news'].mean()*100:.1f}%")

In [ ]:
# Compare IC for rows with news vs without news
test_with_news = test_df[test_df["has_news"] == 1]
test_no_news = test_df[test_df["has_news"] == 0]

ic_with_news = compute_daily_ic(test_with_news)
ic_no_news = compute_daily_ic(test_no_news)

print("Rank IC comparison:")
print(f"  With news:    mean={ic_with_news['ic'].mean():.4f}, std={ic_with_news['ic'].std():.4f}, n_days={len(ic_with_news)}")
print(f"  Without news: mean={ic_no_news['ic'].mean():.4f}, std={ic_no_news['ic'].std():.4f}, n_days={len(ic_no_news)}")

In [ ]:
# Basket returns: compare top-K from news vs no-news subsets
basket_with_news = compute_basket_returns(test_with_news, top_k=20)
basket_no_news = compute_basket_returns(test_no_news, top_k=20)

print("\nBasket performance comparison (top-20):")
print(f"  With news:")
print(f"    Daily excess: {basket_with_news['excess_return'].mean()*100:.3f}%")
print(f"    Sharpe (excess): {basket_with_news['excess_return'].mean() / basket_with_news['excess_return'].std() * np.sqrt(252):.2f}")
print(f"  Without news:")
print(f"    Daily excess: {basket_no_news['excess_return'].mean()*100:.3f}%")
print(f"    Sharpe (excess): {basket_no_news['excess_return'].mean() / basket_no_news['excess_return'].std() * np.sqrt(252):.2f}")

In [ ]:
# Ablation: Score with zeroed-out news embeddings
@torch.no_grad()
def get_scores_no_news(model, df, price_cols, fund_cols, emb_cols, device, batch_size=1024):
    """Get model scores with news embeddings zeroed out."""
    model.eval()
    dataset = PointwiseDataset(df, price_cols, fund_cols, emb_cols)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    scores = []
    for batch in tqdm(loader, desc="Scoring (no news)", leave=False):
        price = batch["price"].to(device)
        fund = batch["fund"].to(device)
        # Zero out embeddings
        emb = torch.zeros_like(batch["emb"]).to(device)
        
        score = model(price, fund, emb)
        scores.append(score.cpu().numpy())
    
    return np.concatenate(scores)

# Score with ablated news
test_df["score_no_news"] = get_scores_no_news(model, test_df, price_feat_cols, fund_feat_cols, emb_cols, device)

# Compare IC
ic_full = compute_daily_ic(test_df)  # Uses "score" column
test_df_ablated = test_df.copy()
test_df_ablated["score"] = test_df_ablated["score_no_news"]
ic_ablated = compute_daily_ic(test_df_ablated)

print("\nAblation study (zeroed news embeddings):")
print(f"  Full model IC:    {ic_full['ic'].mean():.4f}")
print(f"  No-news model IC: {ic_ablated['ic'].mean():.4f}")
print(f"  Delta:            {(ic_full['ic'].mean() - ic_ablated['ic'].mean()):.4f}")

In [ ]:
# Basket performance: full model vs ablated model
basket_full = compute_basket_returns(test_df, top_k=20)
basket_ablated = compute_basket_returns(test_df_ablated, top_k=20)

print("\nBasket performance: Full vs Ablated model:")
print(f"  Full model:")
print(f"    Daily excess: {basket_full['excess_return'].mean()*100:.3f}%")
print(f"    Sharpe (excess): {basket_full['excess_return'].mean() / basket_full['excess_return'].std() * np.sqrt(252):.2f}")
print(f"  Ablated (no news):")
print(f"    Daily excess: {basket_ablated['excess_return'].mean()*100:.3f}%")
print(f"    Sharpe (excess): {basket_ablated['excess_return'].mean() / basket_ablated['excess_return'].std() * np.sqrt(252):.2f}")

In [ ]:
# Focus on rows WITH news: does the model do better with real embeddings vs zeros?
test_news_only = test_df[test_df["has_news"] == 1].copy()

# Full model scores (already have)
ic_news_full = compute_daily_ic(test_news_only)

# Ablated scores for news rows
test_news_ablated = test_news_only.copy()
test_news_ablated["score"] = test_news_only["score_no_news"]
ic_news_ablated = compute_daily_ic(test_news_ablated)

print("\nNews rows only - Full vs Ablated:")
print(f"  Full model IC:    {ic_news_full['ic'].mean():.4f}")
print(f"  Ablated model IC: {ic_news_ablated['ic'].mean():.4f}")
print(f"  Delta (news benefit): {(ic_news_full['ic'].mean() - ic_news_ablated['ic'].mean()):.4f}")

# Is the delta significant?
from scipy.stats import ttest_rel
if len(ic_news_full) == len(ic_news_ablated):
    merged_ic = ic_news_full.merge(ic_news_ablated, on="date", suffixes=("_full", "_ablated"))
    t_stat, p_val = ttest_rel(merged_ic["ic_full"], merged_ic["ic_ablated"])
    print(f"  Paired t-test: t={t_stat:.2f}, p={p_val:.4f}")